In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm

# Velocity threshold in pixels per nanosecond (tune this!)
VELOCITY_THRESHOLD = 1e-5  # px/ns (adjust based on resolution/frame rate)
MIN_FIXATION_DURATION = 100_000_000  # 100 ms in ns


In [ ]:
# Set project structure
project_root = Path.cwd()
day_dir = project_root / "DayTime_Exp_EyeTrackingDevice"
night_dir = project_root / "NightTime_Exp_EyeTrackingDevice"


In [ ]:
def load_all_gaze_files(base_dir):
    gaze_data = {}
    participant_folders = list(base_dir.iterdir())

    for participant_folder in tqdm(participant_folders, desc=f"Loading data from {base_dir.name}"):
        gaze_path = participant_folder / "gaze_positions.csv"
        if gaze_path.exists():
            df = pd.read_csv(gaze_path)
            gaze_data[participant_folder.name] = df
    return gaze_data

# Load daytime and nighttime data with progress bars
day_gaze_data = load_all_gaze_files(day_dir)
night_gaze_data = load_all_gaze_files(night_dir)

## I-VT (Velocity-Threshold Identification) Fixation Detection Function

We're using the I-VT (Identification by Velocity Threshold) algorithm to detect eye fixations from raw gaze data collected during the cycling experiments. This method classifies fixations based on the speed of eye movements between consecutive gaze points.

In simple terms, if the gaze is moving slowly enough (below a predefined velocity threshold) for a minimum duration (e.g., 100 milliseconds), it's considered a fixation. Faster movements are classified as saccades (rapid eye shifts).

For each detected fixation, we also calculate the spatial dispersion, the area covered by the gaze points during that fixation, by measuring the spread of gaze positions. This fixation area provides insight into how concentrated or spread out a cyclist’s visual attention is at different moments.

By applying this same method to both daytime and nighttime gaze data, we ensure a consistent and comparable analysis of visual attention and fixation characteristics across lighting conditions, independent of the original fixation data availability.

In [ ]:
def compute_fixations_ivt(df, velocity_threshold=VELOCITY_THRESHOLD, min_duration=MIN_FIXATION_DURATION):
    """
    Detects fixations using I-VT algorithm and computes fixation spatial dispersion.

    Parameters
    ----------
    df : pandas.DataFrame
        Input gaze data with the following columns:
        - 'timestamp [ns]': Timestamp of source image frame in nanoseconds (ns)
        - 'gaze x [px]': Gaze x-coordinate in world image frame in pixels (px)
        - 'gaze y [px]': Gaze y-coordinate in world image frame in pixels (px)

    velocity_threshold : float, optional
        Velocity threshold in pixels per nanosecond. Gaze movements below this threshold
        are considered part of a fixation. Default is 1e-5.

    min_duration : int, optional
        Minimum fixation duration in nanoseconds (ns). Default is 100,000,000 ns (100 ms).

    Returns
    -------
    pandas.DataFrame
        DataFrame of detected fixations with columns:
        - 'start_time_ns': Fixation start time in nanoseconds (ns)
        - 'end_time_ns': Fixation end time in nanoseconds (ns)
        - 'duration_ns': Fixation duration in nanoseconds (ns)
        - 'fixation_x': Mean fixation x-coordinate in pixels (px)
        - 'fixation_y': Mean fixation y-coordinate in pixels (px)
        - 'dispersion_x': Spatial dispersion in x-direction (width) in pixels (px)
        - 'dispersion_y': Spatial dispersion in y-direction (height) in pixels (px)
        - 'fixation_area': Approximate fixation area as dispersion_x * dispersion_y in pixels squared (px²)

    Notes
    -----
    - The fixation coordinates, dispersion, and area are based on pixel coordinates from the world image frame.
    - Timestamps are maintained in nanoseconds as provided in the raw data.
    - To convert fixation areas to real-world units (e.g., square meters), calibration data linking pixels to spatial distances is required.
    """
    df = df[['timestamp [ns]', 'gaze x [px]', 'gaze y [px]']].dropna().reset_index(drop=True)
    
    fixations = []
    start_idx = 0
    in_fixation = False

    for i in range(1, len(df)):
        dx = df.loc[i, 'gaze x [px]'] - df.loc[i-1, 'gaze x [px]']
        dy = df.loc[i, 'gaze y [px]'] - df.loc[i-1, 'gaze y [px]']
        dt = df.loc[i, 'timestamp [ns]'] - df.loc[i-1, 'timestamp [ns]']
        if dt == 0:
            continue

        velocity = np.sqrt(dx**2 + dy**2) / dt

        if velocity < velocity_threshold:
            if not in_fixation:
                in_fixation = True
                start_idx = i - 1
        else:
            if in_fixation:
                in_fixation = False
                fixation = df.iloc[start_idx:i]
                duration = fixation['timestamp [ns]'].iloc[-1] - fixation['timestamp [ns]'].iloc[0]
                if duration >= min_duration:
                    fixation_x = fixation['gaze x [px]'].mean()
                    fixation_y = fixation['gaze y [px]'].mean()
                    dispersion_x = fixation['gaze x [px]'].max() - fixation['gaze x [px]'].min()
                    dispersion_y = fixation['gaze y [px]'].max() - fixation['gaze y [px]'].min()
                    fixation_area = dispersion_x * dispersion_y

                    fixations.append({
                        'start_time_ns': fixation['timestamp [ns]'].iloc[0],
                        'end_time_ns': fixation['timestamp [ns]'].iloc[-1],
                        'duration_ns': duration,
                        'fixation_x': fixation_x,
                        'fixation_y': fixation_y,
                        'dispersion_x': dispersion_x,
                        'dispersion_y': dispersion_y,
                        'fixation_area': fixation_area
                    })

    # Handle fixation at the end
    if in_fixation:
        fixation = df.iloc[start_idx:]
        duration = fixation['timestamp [ns]'].iloc[-1] - fixation['timestamp [ns]'].iloc[0]
        if duration >= min_duration:
            fixation_x = fixation['gaze x [px]'].mean()
            fixation_y = fixation['gaze y [px]'].mean()
            dispersion_x = fixation['gaze x [px]'].max() - fixation['gaze x [px]'].min()
            dispersion_y = fixation['gaze y [px]'].max() - fixation['gaze y [px]'].min()
            fixation_area = dispersion_x * dispersion_y

            fixations.append({
                'start_time_ns': fixation['timestamp [ns]'].iloc[0],
                'end_time_ns': fixation['timestamp [ns]'].iloc[-1],
                'duration_ns': duration,
                'fixation_x': fixation_x,
                'fixation_y': fixation_y,
                'dispersion_x': dispersion_x,
                'dispersion_y': dispersion_y,
                'fixation_area': fixation_area
            })

    return pd.DataFrame(fixations)


In [ ]:
# Create output dicts
day_fixations = {}
night_fixations = {}

# Process daytime data with progress bar
for participant, df in tqdm(day_gaze_data.items(), desc="Processing Daytime Fixations"):
    day_fixations[participant] = compute_fixations_ivt(df)

# Process nighttime data with progress bar
for participant, df in tqdm(night_gaze_data.items(), desc="Processing Nighttime Fixations"):
    night_fixations[participant] = compute_fixations_ivt(df)


In [ ]:
output_dir = project_root / "Fixation_Results"
output_dir.mkdir(exist_ok=True)

for participant, fixation_df in day_fixations.items():
    fixation_df.to_csv(output_dir / f"{participant}_Fixations.csv", index=False)

for participant, fixation_df in night_fixations.items():
    fixation_df.to_csv(output_dir / f"{participant}_Fixations.csv", index=False)
